# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [16]:
w = data[data.race=='w']['call']
b = data[data.race=='b']['call']

In [17]:
# number of callbacks for white-sounding names
white_callback = sum(data[data.race=='w'].call)

In [18]:
# number of callbacks for black-sounding names
black_callback = sum(data[data.race=='b'].call)

In [19]:
len(w)

2435

In [20]:
len(b)

2435

# Question 1
Yes, the central limit theorem theory does apply. I will apply a permutation test.

# Question 2
The null hypothesis is, that the "race-soundingness" of the name does not influence the probabiltity of a callback.
The alternative Hypothesis is that it does.

In [ ]:
# Your solution to Q3 here

# Question 3 
### Bootstrap approach below

In [24]:
difference = white_callback - black_callback

In [25]:
difference

78.0

In [32]:
np.random.seed(8)
differences = np.empty(10000)
for i in range(10000):
    all_perm = np.random.permutation(pd.concat([w, b]))
    whites = all_perm[:len(w)]
    blacks = all_perm[len(w):]
    differences[i] = sum(whites) - sum(blacks)


In [33]:
differences.max()

70.0

In [34]:
p = len(differences[differences > difference])/10000

In [62]:
p

1.0

In [35]:
confidence_interval = np.percentile(differences, [2.5,97.5])

### Frequentist approach below

In [50]:
prob_b = black_callback/len(b)
prob_b*len(b)

157.0

In [51]:
prob_w = white_callback/len(w)
prob_w*len(w)

235.0

In [75]:
prob_both = (black_callback + white_callback)/(len(b)+len(w))
prob_both

0.08049281314168377

As the probability of both groups to get a callback multiplied by the sample sizes is large enough, I can use a normal approximation for the binomial distribution

In [78]:
mean_b = prob_b*len(b)
mean_w = prob_w*len(w)
mean_both = prob_both*(len(b)+len(w))

std_b = prob_b*len(b)*(1-prob_b)
std_w = prob_w*len(w)*(1-prob_w)
std_both = prob_both*(len(w)+len(b))*(1-prob_both)

In [61]:
n = len(b)
standard_error_b = std_b/np.sqrt(n)
standard_error_w = std_w/np.sqrt(n)
difference_se = np.sqrt(standard_error_b**2 + standard_error_w**2)
z = (mean_w-mean_b)/difference_se
p = stats.norm.cdf(z)

1.0

In [71]:
print(z,p)

14.908528131735105 1.0


In [68]:
confidence_95 = stats.norm.ppf(0.025), stats.norm.ppf(0.975)

In [72]:
t = stats.t.ppf(0.95,len(b)+len(w)-1)

In [79]:
margin_of_error = t*std_both/np.sqrt(len(w)+len(b))

8.497410694182433

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# Question 4
Both the Bootstrap and the frequentist test confirm that the null hypothesis can be clearly rejected at any confidence level and that most likely people with black-sounding names are less likely to receive a callback.

# Question 5
No, of course it doesn't say that, as its the only factor that was analyzed by the study. In order to know if its the most important factor, all possible factors would have to be tested (which would be impossible, but one could analyse more factors).